# Vector Timeseries Application

## Imports and map initialisation

In [1]:
import folium
import os
import psycopg2
from folium import plugins
print(folium.__version__)
import geopandas as gpd
import pandas as pd
import json

0.10.1


## 4 Cities

In [3]:
## I choose dbname appropriately
conn = psycopg2.connect(database="mydb",
  user="user",
  host="localhost",
  password="user")
df = gpd.read_postgis(
     'select gid, mun, heat, water, geom, St_AsText(St_Centroid(geom)) as center from final', 
    conn, geom_col='geom')
cities = gpd.read_postgis(
     'select gid, mun, heat, water, geom, St_AsText(St_Centroid(geom)) as center,timeseries from final where timeseries is not null', 
    conn, geom_col='geom')

In [4]:
fil_center = cities.iloc[0].geom.centroid
dra_center = cities.iloc[1].geom.centroid
cha_center = cities.iloc[2].geom.centroid
ath_center = cities.iloc[3].geom.centroid

### Vincent timeseries layers from geoserver
First, I used **OWSlib** for a WFS call to geoserver in order to download the timeseries tables in .csv format.  

Download function

In [5]:
#WFS, OWSlib
from owslib.wfs import WebFeatureService
wfs = WebFeatureService(url='http://127.0.0.1:8082/geoserver/wfs', version='1.1.0')
def getFeature(layer, bbox=None, workspace="geonode", format="json"):
    if bbox:
        response = wfs.getfeature(typename=workspace+":"+layer, outputFormat=format, bbox=bbox)
    else:
        response = wfs.getfeature(typename=workspace+":"+layer, outputFormat=format)
    with open(os.path.join("wfs", layer+"."+format), 'wb') as f:
        while True:
            data = response.read(100)
            if not data:
                break
            f.write(data)
    name=f.name
    f.close
    return (name)

Download process

In [6]:
layer = "fil_month"
fil_month = pd.read_csv(getFeature(layer, format="csv"))[["id","date","year","month","kt","t2m","prectot"]]
fil_month.head()

,id,date,year,month,kt,t2m,prectot
0,1,1981-01-01,1981,1,0.572246,6.720000,183.160004
1,2,1981-02-01,1981,2,0.572246,8.810000,77.010002
2,3,1981-03-01,1981,3,0.572246,12.450000,16.270000
3,4,1981-04-01,1981,4,0.572246,14.720000,47.910000
4,5,1981-05-01,1981,5,0.572246,17.629999,40.299999


Second, I load the csvs to variables and process them for popups

In [7]:
#filiatra
fil_month = pd.read_csv("wfs/fil_month.csv")
fil_year = pd.read_csv("wfs/fil_year.csv")
fil_year.index = fil_year.year
kt_fil_year = fil_year[["KT"]]

#drama
dra_month = pd.read_csv("wfs/dra_month.csv")
dra_year = pd.read_csv("wfs/dra_year.csv")
dra_year.index = dra_year.year
kt_dra_year = dra_year[["KT"]]

#chania
cha_month = pd.read_csv("wfs/cha_month.csv")
cha_year = pd.read_csv("wfs/cha_year.csv")
cha_year.index = cha_year.year
kt_cha_year = cha_year[["KT"]]

#athens
ath_month = pd.read_csv("wfs/ath_month.csv")
ath_year = pd.read_csv("wfs/ath_year.csv")
ath_year.index = ath_year.year
kt_ath_year = ath_year[["KT"]]

In [8]:
#ath_year_dict = {"index": list(ath_year.year.values), "T2M": list(ath_year.T2M.values)} #"KT": list(ath_year.KT.values)}
#KT_yearly = vt.Data.from_mult_iters(index=list(ath_year.year.values), KT=list(ath_year.KT.values), idx='index')
#KT_yearly = vt.Data.from_pandas(ath_year, key_on="year")
#type(ath_year_dict)

### Vincent objects to parse them into Vega as json
I use vincent library to prepare the popup thumbnails.

In [9]:
import vincent
import json

x= 'Year'
y= 'Insolation Clearness Index'

fil = vincent.StackedArea(kt_fil_year)
fil.axis_titles(x=x, y=y)
fil.height=200
fil.width=400

dra = vincent.StackedArea(kt_dra_year)
dra.axis_titles(x=x, y=y)
dra.height=200
dra.width=400

cha = vincent.StackedArea(kt_cha_year)
cha.axis_titles(x=x, y=y)
cha.height=200
cha.width=400

vincent.core.initialize_notebook()
ath = vincent.StackedArea(kt_ath_year)
ath.axis_titles(x=x, y=y)
#line.legend(title='Categories')
ath.display()
ath.height=200
ath.width=400

## Map

In [10]:
cities

,gid,mun,heat,water,geom,center,timeseries
0,2128,D. FILIATRVN,4.4269,0.5526,"MULTIPOLYGON (((21.59604 37.21353, 21.59646 37...",POINT(21.6020709743274 37.1453985735083),"{'time': [['Main OPeNDAP Requests:', 3.4420318..."
1,3627,D. DRAMAS,37.3206,2.8256,"MULTIPOLYGON (((24.21158 41.25380, 24.21169 41...",POINT(24.155463651569 41.1650850380416),"{'time': [['Main OPeNDAP Requests:', 3.7805459..."
2,5719,D. XANIVN,122.0812,16.2495,"MULTIPOLYGON (((24.05162 35.53692, 24.05241 35...",POINT(24.0362659071855 35.5119914729891),"{'time': [['Main OPeNDAP Requests:', 4.1280920..."
3,5782,D. AUHNAIVN,646.9412,75.1725,"MULTIPOLYGON (((23.74365 38.03364, 23.74432 38...",POINT(23.7360363484277 37.9888765430097),"{'time': [['Main OPeNDAP Requests:', 3.4559729..."


## Timeseries regions final map

In [12]:
# Map Tiles
m = folium.Map([37, 22], tiles='cartodbpositron', zoom_start=5)

# WMS for custom background from geoserver
from owslib.wms import WebMapService
url = 'http://localhost:8082/geoserver/ows'
wms = WebMapService(url, version='1.1.1')


folium.raster_layers.WmsTileLayer(
    url=url,
    layers='geonode:ne_10m_admin_0_countries',
    name='countries',
    fmt='image/png',
    overlay=False,
    control=True,
).add_to(m)

## 4 cities
mc = folium.plugins.MarkerCluster(name="Municipality Markers")

#athens
mc.add_child(
    folium.Marker(
        location = [fil_center.y, fil_center.x],
        tooltip='Click me to view Insolation Clearness Index timeseries for Filiatra!',
        popup=folium.Popup(max_width=450).add_child(
               folium.Vega(ath.to_json(), width=450, height=250))
    )
)

#filiatra
mc.add_child(
    folium.Marker(
        location = [dra_center.y, dra_center.x],
        tooltip='Click me to view Insolation Clearness Index timeseries for Drama!',
        popup=folium.Popup(max_width=450).add_child(
           folium.Vega(fil.to_json(), width=450, height=250))
    )
)

#chania
mc.add_child(
    folium.Marker(
        location = [cha_center.y, cha_center.x],
        tooltip='Click me to view Insolation Clearness Index timeseries for Chania!',
        popup=folium.Popup(max_width=450).add_child(
           folium.Vega(cha.to_json(), width=450, height=250))
    )
)

#drama
mc.add_child(
    folium.Marker(
        location = [ath_center.y, ath_center.x],
        tooltip='Click me to view Insolation Clearness Index timeseries for Athens!',
        popup=folium.Popup(max_width=450).add_child(
           folium.Vega(dra.to_json(), width=450, height=250))
        
    )
)
m.add_child(mc)

# vector data polygons

locations=[]
for city in range(len(cities)):
    a = []
    for polygon in cities.geom[city]:
        a.extend(polygon.exterior.coords[:-1])
    points = [[t[1], t[0]] for t in a]
    locations.append(points)
locations
    
poly_group = folium.FeatureGroup(name='Municipality Polygons')

poly = folium.Polygon(
    locations=locations[0],
    color='blue',
    weight=6,
    fill_color='red',
    fill_opacity=0.5,
    fill=True,
    popup='',
    tooltip='Filiatra municipality bounds',
).add_to(poly_group)

poly = folium.Polygon(
    locations=locations[1],
    color='blue',
    weight=6,
    fill_color='red',
    fill_opacity=0.5,
    fill=True,
    popup='Drama municipality bounds',
    tooltip='Drama!',
).add_to(poly_group)

poly = folium.Polygon(
    locations=locations[2],
    color='blue',
    weight=6,
    fill_color='red',
    fill_opacity=0.5,
    fill=True,
    popup='Chania municipality bounds',
    tooltip='Chania!',
).add_to(poly_group)

poly = folium.Polygon(
    locations=locations[3],
    color='blue',
    weight=6,
    fill_color='red',
    fill_opacity=0.5,
    fill=True,
    popup='Athens municipality bounds',
    tooltip='Athens!',
).add_to(poly_group)

m.add_child(poly_group)

# Full screen
plugins.Fullscreen(
    position='topright',
    title='Expand me',
    title_cancel='Exit me',
    force_separate_button=True).add_to(m)

#Layer control
folium.LayerControl().add_to(m)

# ScrollZoomToggler
plugins.ScrollZoomToggler().add_to(m)

# Measurements
plugins.MeasureControl().add_to(m)

m

## Html output save

In [13]:
m.save(os.path.join('results', 'timeseries_regions.html'))